In [1]:
import pynq
from pynq import Overlay, MMIO
from pynq.lib.dma import DMA
import numpy as np
from pynq import allocate

overlay = Overlay("design_1.bit")
dma = overlay.axi_dma_0

In [2]:
import numpy as np

X_test = np.load('X_test.npy')
y_test = np.load('y_test.npy')

In [3]:
for i in range(20):
    print(i)
    print(X_test[i])
    print(y_test[i])

0
[-0.11950312  0.40616292 -1.04058613 -0.82473132 -0.75533621 -0.58097338
  1.98691601  1.53806275  1.98691601  0.6312378   0.38356148 -0.20130531
  1.05976638  0.40865762 -1.01995561 -0.18016747]
[0. 1. 0. 0. 0.]
1
[ 0.30887296  0.2271202  -1.15605569 -0.84608115 -1.03461424 -0.61983525
  0.18320819  0.59584307  0.18320819  1.70236643  2.04099658  2.42276296
  1.52444966  1.86744644 -1.23200286 -1.19496887]
[0. 1. 0. 0. 0.]
2
[-1.26550208  0.66753284  1.40874527  1.40118472  1.57752933  1.29271974
  0.12350547  0.02704562  0.12350547  0.32859339  0.52060233  0.73334179
  0.55196236  0.58246973  1.3092331   1.75718066]
[0. 0. 1. 0. 0.]
3
[-1.06506184  0.68234668  2.17392149  2.234143    2.05064568  2.35187808
 -0.13290398  0.22828887 -0.13290398  0.90665935  1.110763    1.61273002
  0.91171934  1.39581995  1.81261011  0.97301594]
[0. 0. 1. 0. 0.]
4
[-1.52057416  0.87718789  2.42401652  2.76920197  1.9946634   1.78061237
 -0.35790072 -0.30510732 -0.35790072  0.52400863  1.05520844  1.1

In [8]:
import numpy as np
from pynq import allocate, MMIO

# Assuming dma is set up and gpio_0_baseaddr points to the base address of the output GPIO
gpio_0_baseaddr = 0x41200000  # Example base address, replace with actual base address
gpio_0 = MMIO(gpio_0_baseaddr, 0x10000)

for i in range(1):
    t = X_test[i]
    n = len(t)
    
    # Convert to appropriate data type for FPGA
    array_t = np.array(t)
    sample = array_t.astype(np.float32)

    # Allocate buffers for DMA
    with allocate(shape=(n,), dtype=np.float32) as in_buffer:
        # Copy data into DMA buffer
        np.copyto(in_buffer, sample)
        
        # Send data to FPGA
        dma.sendchannel.transfer(in_buffer)
        dma.sendchannel.wait()  # Wait for transfer completion

        # After processing, read output from GPIO
        # Assuming each output is a 16-bit value mapped to separate GPIO registers
        results = []
#         for j in range(5):  # 5 outputs of 16 bits each
#             result = gpio_0.read(j * 4)  # Adjust offset if necessary
#             results.append(result)
        
        print(f"Results for sample {i}: {[hex(r) for r in results]}")


KeyboardInterrupt: 

In [27]:
for i in range(10):
    t = X_test[i]
    n = len(t)
#     print(n)
    # array_t = np.array(t)

    # sample = array_t.astype(np.int32)

    array_t = np.array(t)

    sample = array_t.astype(np.float32)
#     print(sample)

    with allocate(shape=(n,), dtype=np.float32) as in_buffer, \
         allocate(shape=(n,), dtype=np.float32) as out_buffer:

        np.copyto(in_buffer,sample)
        dma.sendchannel.transfer(in_buffer)

        dma.sendchannel.wait()
#         print(in_buffer)
        gpio_0_baseaddr = 0x4120_0000
        gpio_0 = MMIO(gpio_0_baseaddr,0x10000)
        result  = gpio_0.read(0x00)
        print(f"result : {hex(result)}")

result : 0x0
result : 0x0
result : 0x0


KeyboardInterrupt: 

result : 0x333
